# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [162]:
import requests

In [163]:
from bs4 import BeautifulSoup

In [164]:
import pandas as pd

In [165]:
# определяем список ключевых слов, которые нам интересны
KEYWORDS = ['python', 'парсинг']

In [167]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id">
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" rel="image_src"/>
<meta content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="uns

In [170]:
# извлекаем посты
posts = soup.find_all('article', class_='post')

In [172]:
# Вариант 1: добавляем извлечение превью текста из постов, чтобы отбирать только посты, где ключевые слова встречаются в превью
df = pd.DataFrame(columns=['date', 'title', 'link'])

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    #post_id = int(post_id.split('_')[-1])
    previews = post.find_all('div', class_='post__body post__body_crop')
    for preview in previews:
            preview_lower = preview.text.lower()
           # ищем вхождение ключевого слова в превью 
            if any([desired in preview_lower for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                date_element = post.find('span', class_='post__time') 
                #print(date_element.text, title_element.text, title_element.attrs.get('href'))
                df.loc[len(df.index)] = [date_element.text, title_element.text, title_element.attrs.get('href')]
df

,date,title,link


In [173]:
# Вариант 2: ключевые слова могут встречасться не только в првеью, но и например в keywords. Если искать ключевое слово везде в пост:
df = pd.DataFrame(columns=['date', 'title', 'link'])

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
   # post_id = int(post_id.split('_')[-1])
    if any([desired in post.text.lower() for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
        title_element = post.find('a', class_='post__title_link')
        date_element = post.find('span', class_='post__time') 
        #print(date_element.text, title_element.text, title_element.attrs.get('href'))
        df.loc[len(df.index)] = [date_element.text, title_element.text, title_element.attrs.get('href')]
df

,date,title,link
0,сегодня в 01:00,Популярность BPM в разных жанрах музыки. Анали...,https://habr.com/ru/post/544540/
1,вчера в 20:09,Новое тестирование фичей в Django 3.2,https://habr.com/ru/company/otus/blog/544880/
2,вчера в 19:33,Нейродайджест: главное из области машинного об...,https://habr.com/ru/post/544894/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [175]:
import time

In [177]:
# проверка
URL = 'https://habr.com/ru/all/'
params = {
    'search_query': 'python'
}
i=1
url = URL+(f'page{i + 1}/')
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Refridgerator/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/47b/c63/a93/47bc63a93bedea659c9bb02b4c610267.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Refridgerator</span>
 </a>
 <span class="post__time">вчера в 18:23</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/544786/">Как писать на ассемблере в 2021 году</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/programming/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Программирование'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Програм

In [179]:
# новости с заданного количества страниц
def get_all_links(url, pages):
    all_refs = []
    for i in range(pages):
        url_page = url+(f'page{i + 1}/')
        res = requests.get(url_page)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        posts = soup.find_all('article', class_='post')
        for post in posts: 
            all_refs.append(post.find('a', class_='post__title_link').get('href'))
    return all_refs

all_links = get_all_links(URL, 3)
all_links

['https://habr.com/ru/company/timeweb/blog/544956/',
 'https://habr.com/ru/post/544540/',
 'https://habr.com/ru/post/544954/',
 'https://habr.com/ru/company/timeweb/blog/544934/',
 'https://habr.com/ru/post/544932/',
 'https://habr.com/ru/company/spbifmo/blog/544922/',
 'https://habr.com/ru/post/544916/',
 'https://habr.com/ru/post/544914/',
 'https://habr.com/ru/company/skillfactory/blog/544846/',
 'https://habr.com/ru/company/audiomania/blog/544686/',
 'https://habr.com/ru/company/otus/blog/544880/',
 'https://habr.com/ru/company/otus/blog/544876/',
 'https://habr.com/ru/post/544390/',
 'https://habr.com/ru/company/otus/blog/544862/',
 'https://habr.com/ru/company/infoculture/blog/544896/',
 'https://habr.com/ru/post/544894/',
 'https://habr.com/ru/company/kaspersky/blog/544888/',
 'https://habr.com/ru/post/544732/',
 'https://habr.com/ru/post/544884/',
 'https://habr.com/ru/company/mvideo/blog/544852/',
 'https://habr.com/ru/post/544786/',
 'https://habr.com/ru/company/JetBrains-edu

In [180]:
# делаем проверку на ключевые слова, выбираем нужные элементы и добавляем в датафрейм

df = pd.DataFrame(columns=['date', 'title', 'link', 'article text'])

for link in all_links:
        res = requests.get(link)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        posts = soup.find_all('article', class_='post')
        for post in posts:
            if any([desired in post.text.lower() for desired in KEYWORDS]):    
                title_element = post.find('span', class_='post__title-text')
                text_element = post.find('div', class_='post__body post__body_full')        
                date_element = post.find('span', class_='post__time') 
                df.loc[len(df.index)] = [date_element.text, title_element.text, link, text_element]
            pass
       
df


,date,title,link,article text
0,сегодня в 01:01,Почему веб-производительность внутренних систе...,https://habr.com/ru/company/timeweb/blog/544956/,"[\n, [Как-то раз у меня был занятный разговор ..."
1,сегодня в 01:00,Популярность BPM в разных жанрах музыки. Анали...,https://habr.com/ru/post/544540/,"[\n, [[<img height=""1917"" src=""https://habrast..."
2,вчера в 21:22,Отслеживание лиц в реальном времени в браузере...,https://habr.com/ru/company/skillfactory/blog/...,"[\n, [[<img height=""600"" src=""https://habrasto..."
3,вчера в 20:09,Новое тестирование фичей в Django 3.2,https://habr.com/ru/company/otus/blog/544880/,"[\n, [[<img height=""439"" src=""https://habrasto..."
4,вчера в 19:43,День открытых данных 2021. Онлайн,https://habr.com/ru/company/infoculture/blog/5...,"[\n, [[<img alt=""image"" src=""https://habrastor..."
5,вчера в 19:33,Нейродайджест: главное из области машинного об...,https://habr.com/ru/post/544894/,"[\n, [[], [], \r\nКак вы знаете, в подборку мы..."
6,вчера в 17:49,Из студентов в преподаватели: интервью с выпус...,https://habr.com/ru/company/JetBrains-educatio...,"[\n, [Продолжаем знакомить вас с выпускниками ..."
7,вчера в 17:43,Архитектура в Django проектах — как выжить,https://habr.com/ru/company/vivid_money/blog/5...,"[\n, [[Думаю, ни для кого не секрет, что в раз..."
8,вчера в 17:21,Как я сократил время загрузки GTA Online на 70%,https://habr.com/ru/post/544764/,"[\n, [[<img alt=""image"" src=""https://habrastor..."
9,вчера в 16:14,Как я сократил время загрузки GTA Online на 70%,https://habr.com/ru/company/itsumma/blog/544830/,"[\n, [GTA Online. Многопользовательская игра, ..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [182]:
import json

In [183]:
URL= 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['xxx@x.ru', 'yyy@y.com']

headers = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}
payload = {
        'emailAddresses': EMAIL
}

In [184]:
res = requests.post(URL, json=payload, headers=headers)
res

<Response [200]>

In [185]:
# превращаем результат в структуру из вложенных словарей
dict_ = json.loads(res.text)
dict_

{'breaches': {'16613': {'breachId': 16613,
   'site': 'verifications.io',
   'recordsCount': 677914246,
   'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n',
   'publishDate': '2019-03-28T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}},
  '17609': {'breachId': 17609,
   'site': 'vk.com',
   'recordsCount': 42536910,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and

In [155]:
# структура содержит 3 словаря
for a,b in dict_.items():
    print(a)

breaches
data
summary


In [153]:
# словарь {id утечки : все детали по утечке}
for a,b in dict_['breaches'].items():
    print(a, b)
    break

16613 {'breachId': 16613, 'site': 'verifications.io', 'recordsCount': 677914246, 'description': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n', 'publishDate': '2019-03-28T00:00:00Z', 'statistics': {'usernames': 0, 'passwords': 0, 'emails': 677914246}}


In [146]:
#  словарь {источник утечки : детали по утечки конкретного емейла} - это лишняя информация, не будем использовать, но можно себя проверить
for a,b in dict_['data'].items():
    print(a, b)

parapa.mail.ru {'xxx@x.ru': [{'breachId': 3176, 'usernameBreached': True, 'passwordBreached': True}]}
verifications.io {'xxx@x.ru': [{'breachId': 16613, 'usernameBreached': True, 'passwordBreached': True}]}
adobe.com {'xxx@x.ru': [{'breachId': 3, 'usernameBreached': True, 'passwordBreached': True}], 'yyy@y.com': [{'breachId': 3, 'usernameBreached': True, 'passwordBreached': True}]}
cdprojektred.com {'xxx@x.ru': [{'breachId': 2961, 'usernameBreached': True, 'passwordBreached': True}]}
azcentral.com {'yyy@y.com': [{'breachId': 17110, 'usernameBreached': True, 'passwordBreached': True}]}
linkedin.com {'yyy@y.com': [{'breachId': 2, 'usernameBreached': True, 'passwordBreached': True}]}
dropbox.com {'yyy@y.com': [{'breachId': 41, 'usernameBreached': True, 'passwordBreached': True}]}
wishbone.io {'yyy@y.com': [{'breachId': 17670, 'usernameBreached': True, 'passwordBreached': True}]}
youku.com {'yyy@y.com': [{'breachId': 3669, 'usernameBreached': True, 'passwordBreached': True}]}
myheritage.co

In [151]:
# словарь {почта : список id утечек}
for a,b in dict_['summary'].items():
    print(a, b)


xxx@x.ru {'breaches': [16613, 17609, 3176, 12, 3, 3164, 2961, 15]}
yyy@y.com {'breaches': [17110, 17670, 13094, 37177, 16768, 41, 2, 3587, 17009, 3, 13662, 3520, 15, 3669]}


In [186]:
# нужен формат <почта> - <дата утечки> - <источник утечки> - <описание утечки>
# финальная функция:

def list_breaches():
    """
    функция проходит по словарю summary, берет для каждого email по очереди id каждой утечки
    и подтягивает по этому id детали из соответвующего словаря в словаре  breaches,
    добавляет строку в дата фрейм
    """
    
    df = pd.DataFrame(columns=['email', 'breach date', 'breach source', 'breach description'])

    for email, breaches in dict_['summary'].items():
        for breach_id in breaches['breaches']:
            breach_date = dict_['breaches'][str(breach_id)]['publishDate']
            breach_source = dict_['breaches'][str(breach_id)]['site']
            breach_description = dict_['breaches'][str(breach_id)]['description']

            df.loc[len(df.index)] = [email, breach_date, breach_source, breach_description]
    return df
    

In [187]:
# применяем функцию
list_breaches()

,email,breach date,breach source,breach description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
9,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`